In [4]:
import scipy.stats as stats
import math
from yahooquery import Ticker
import pandas as pd


def black_scholes(S, K, T, r, sigma):
    d1 = (math.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    call = S * stats.norm.cdf(d1) - K * math.exp(-r * T) * stats.norm.cdf(d2)
    put = K * math.exp(-r * T) * stats.norm.cdf(-d2) - S * stats.norm.cdf(-d1)
    return call, put


In [17]:
stk = Ticker("AAPL")

In [18]:
option_chain = stk.option_chain
options_df = option_chain.reset_index()
calls_df = options_df[options_df['optionType'] == 'calls']
puts_df = options_df[options_df['optionType'] == 'puts']

calls_df.head()



,symbol,expiration,optionType,contractSymbol,strike,currency,lastPrice,change,percentChange,volume,openInterest,bid,ask,contractSize,lastTradeDate,impliedVolatility,inTheMoney
0,AAPL,2024-11-22,calls,AAPL241122C00100000,100.0,USD,127.30,-1.589996,-1.233607,2.0,10.0,126.40,126.85,REGULAR,2024-11-20 17:13:57,3.796876,True
1,AAPL,2024-11-22,calls,AAPL241122C00125000,125.0,USD,99.89,0.000000,0.000000,0.0,2.0,101.50,101.95,REGULAR,2024-11-15 17:46:23,2.328129,True
2,AAPL,2024-11-22,calls,AAPL241122C00130000,130.0,USD,99.15,0.000000,0.000000,3.0,4.0,96.45,96.85,REGULAR,2024-11-18 17:59:33,2.664066,True
3,AAPL,2024-11-22,calls,AAPL241122C00135000,135.0,USD,92.99,0.000000,0.000000,3.0,4.0,91.35,91.85,REGULAR,2024-11-08 20:24:06,2.500004,True
4,AAPL,2024-11-22,calls,AAPL241122C00140000,140.0,USD,84.82,0.000000,0.000000,8.0,8.0,86.50,86.90,REGULAR,2024-11-15 15:50:11,0.500005,True


In [19]:
calls_df['mid_price'] = (calls_df['ask'] + calls_df['bid']) / 2
calls_df = calls_df[calls_df['mid_price'] > 1]
calls_df.head()

/var/folders/s1/9bykr2ps10l_lqbqpvjd6ntr0000gn/T/ipykernel_18570/163983822.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calls_df['mid_price'] = (calls_df['ask'] + calls_df['bid']) / 2


,symbol,expiration,optionType,contractSymbol,strike,currency,lastPrice,change,percentChange,volume,openInterest,bid,ask,contractSize,lastTradeDate,impliedVolatility,inTheMoney,mid_price
0,AAPL,2024-11-22,calls,AAPL241122C00100000,100.0,USD,127.30,-1.589996,-1.233607,2.0,10.0,126.40,126.85,REGULAR,2024-11-20 17:13:57,3.796876,True,126.625
1,AAPL,2024-11-22,calls,AAPL241122C00125000,125.0,USD,99.89,0.000000,0.000000,0.0,2.0,101.50,101.95,REGULAR,2024-11-15 17:46:23,2.328129,True,101.725
2,AAPL,2024-11-22,calls,AAPL241122C00130000,130.0,USD,99.15,0.000000,0.000000,3.0,4.0,96.45,96.85,REGULAR,2024-11-18 17:59:33,2.664066,True,96.650
3,AAPL,2024-11-22,calls,AAPL241122C00135000,135.0,USD,92.99,0.000000,0.000000,3.0,4.0,91.35,91.85,REGULAR,2024-11-08 20:24:06,2.500004,True,91.600
4,AAPL,2024-11-22,calls,AAPL241122C00140000,140.0,USD,84.82,0.000000,0.000000,8.0,8.0,86.50,86.90,REGULAR,2024-11-15 15:50:11,0.500005,True,86.700


In [11]:
# Get current stock price
current_price = stk.price['AAPL']['regularMarketPrice']

# Calculate days to expiration and convert to years
calls_df['days_to_expiry'] = pd.to_datetime(calls_df['expiration']) - pd.Timestamp.now()
calls_df['T'] = calls_df['days_to_expiry'].dt.days / 365

# Use 10-year Treasury rate as risk-free rate (approximate)
r = 0.045

# Calculate theoretical prices using Black-Scholes
calls_df['bs_price'] = calls_df.apply(
    lambda row: black_scholes(
        S=current_price,
        K=row['strike'], 
        T=row['T'],
        r=r,
        sigma=row['impliedVolatility']
    )[0],  # Index 0 gets call price
    axis=1
)

# Compare market vs model prices
calls_df['price_diff'] = calls_df['mid_price'] - calls_df['bs_price']

# Display results
print("Black-Scholes vs Market Prices:")
print(calls_df[['strike', 'mid_price', 'bs_price', 'price_diff']].head())


Black-Scholes vs Market Prices:
   strike  mid_price    bs_price  price_diff
0   100.0    127.025  126.504818    0.520182
1   125.0    101.950  101.507661    0.442339
2   130.0     96.950   96.508414    0.441586
3   135.0     92.000   91.509911    0.490089
4   140.0     87.000   86.510738    0.489262
